In [83]:
import numpy as np
import pandas as pd
import torch as t
import torch.nn as nn
data = pd.read_csv("compas-scores-two-years.csv")
data.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


In [84]:
pd.unique(data['race'])

array(['Other', 'African-American', 'Caucasian', 'Hispanic',
       'Native American', 'Asian'], dtype=object)

In [85]:
#filter for only African-American and Caucasian
data_2race = data.loc[(data['race']=='African-American') | (data['race']=='Caucasian')]
pd.unique(data_2race['race'])

array(['African-American', 'Caucasian'], dtype=object)

In [86]:
#select a few columns to be used in the following functions
selected_columns = ["two_year_recid","sex","age","race","v_decile_score","priors_count.1",]
data_filtered = data_2race[selected_columns]
data_filtered.head()

,two_year_recid,sex,age,race,v_decile_score,priors_count.1
1,1,Male,34,African-American,1,0
2,1,Male,24,African-American,3,4
3,0,Male,23,African-American,6,1
6,1,Male,41,Caucasian,2,14
8,0,Female,39,Caucasian,1,0


In [87]:
#check for missing values
data_filtered.isna().sum()

two_year_recid    0
sex               0
age               0
race              0
v_decile_score    0
priors_count.1    0
dtype: int64

In [88]:
pd.unique(data_filtered['sex'])

array(['Male', 'Female'], dtype=object)

In [89]:
#replace all categorical values to integer values
data_filtered["race"].replace(['African-American', 'Caucasian'],[0, 1], inplace=True)
data_filtered["sex"].replace(['Male', 'Female'],[0, 1], inplace=True)

C:\Users\smral\anaconda3\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [90]:
data_filtered.head()

,two_year_recid,sex,age,race,v_decile_score,priors_count.1
1,1,0,34,0,1,0
2,1,0,24,0,3,4
3,0,0,23,0,6,1
6,1,0,41,1,2,14
8,0,1,39,1,1,0


In [91]:
#baseline model
#logistic regression
import numpy as np

# Define the original dataset
X = data_filtered[["age","sex","v_decile_score","priors_count.1"]]
y = data_filtered[['two_year_recid']]

# Define the sensitive attribute
s = data_filtered[["race"]]

# Define the logistic regression model
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()

# Train the model on the original dataset
clf.fit(X, y)

# Evaluate the model on the original dataset
print("Original dataset:")
print("Accuracy:", clf.score(X, y))

Original dataset:
Accuracy: 0.6721951219512196


C:\Users\smral\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [93]:
#preprocess data, split the data into African-American Group and Caucassian Group
from sklearn.model_selection import train_test_split

data_filtered_afram = data_filtered[data_filtered['race']==0]
data_filtered_cau = data_filtered[data_filtered['race']==1]

X_afram = data_filtered_afram[["age","sex","v_decile_score","priors_count.1","race"]]
y_afram = data_filtered_afram[['two_year_recid']]

X_cau = data_filtered_cau[["age","sex","v_decile_score","priors_count.1","race"]]
y_cau = data_filtered_cau[['two_year_recid']]

In [94]:
X_train_afram, X_test_afram, y_train_afram, y_test_afram = train_test_split(X_afram, y_afram, test_size=0.2)
X_train_cau, X_test_cau, y_train_cau, y_test_cau = train_test_split(X_cau, y_cau, test_size=0.2)

In [95]:
def tensorX(X):
    return t.tensor(np.array(X)).to(t.float32)

def tensorY(X,Y):
    return t.from_numpy(np.array(Y).astype('float32')).reshape(X.shape[0], 1)

In [96]:
X_train_afram = tensorX(X_train_afram)
X_test_afram = tensorX(X_test_afram)
X_train_cau = tensorX(X_train_cau)
X_test_cau = tensorX(X_test_cau)

y_train_afram = tensorY(X_train_afram,y_train_afram)
y_test_afram = tensorY(X_test_afram,y_test_afram)
y_train_cau = tensorY(X_train_cau,y_train_cau)
y_test_cau = tensorY(X_test_cau,y_test_cau)

In [97]:
class LogisticRegression(nn.Module):
    def __init__(self,data):
        super(LogisticRegression, self).__init__()
        self.w = nn.Linear(data.shape[1], out_features=1, bias=True)
        self.sigmod = nn.Sigmoid()
    def forward(self, x):
        w = self.w(x)
        output = self.sigmod(w)
        return output

In [98]:
#Define the Prejudice Remover Regularizer Loss Function
class PRLoss():
    def __init__(self, eta=1.0):
        super(PRLoss, self).__init__()
        self.eta = eta       
        
    def forward(self,output_a,output_c):
        N_a = t.tensor(output_a.shape[0])
        N_c   = t.tensor(output_c.shape[0])
        Dxisi = t.stack((N_a,N_c),axis=0)
        # Pr[y|s]
        y_pred_a = t.sum(output_a)
        y_pred_c   = t.sum(output_c)
        P_ys = t.stack((y_pred_a,y_pred_c),axis=0) / Dxisi
        # Pr[y]
        P = t.cat((output_a,output_c),0)
        P_y = t.sum(P) / (X_train_afram.shape[0]+X_train_cau.shape[0])
        # P(siyi)
        P_s1y1 = t.log(P_ys[1]) - t.log(P_y)
        P_s1y0 = t.log(1-P_ys[1]) - t.log(1-P_y)
        P_s0y1 = t.log(P_ys[0]) - t.log(P_y)
        P_s0y0 = t.log(1-P_ys[0]) - t.log(1-P_y)
        # PI
        PI_s1y1 = output_a * P_s1y1
        PI_s1y0 =(1- output_a) * P_s1y0
        PI_s0y1 = output_c * P_s0y1
        PI_s0y0 = (1- output_c )* P_s0y0
        PI = t.sum(PI_s1y1) + t.sum(PI_s1y0) + t.sum(PI_s0y1) + t.sum(PI_s0y0)
        PI = self.eta * PI
        return PI

In [99]:
#Define the Accuracy and Caliberation function
#Accuracy is the average of correctly predicted labels of two groups
#Caliberation is the difference of accuracy bewtween the two groups
def metrics_cal(Model_a,Model_c, X_a, y_a, X_c, y_c):
    y_pred_a = (Model_a(X_a) >= 0.5)
    y_pred_c = (Model_c(X_c) >= 0.5)
    accuracy_a  = t.sum(y_pred_a.flatten() == y_a.flatten()) / y_a.shape[0]
    accuracy_c  = t.sum(y_pred_c.flatten() == y_c.flatten()) / y_c.shape[0]
    accuracy = (accuracy_a + accuracy_c) / 2
    cali = t.abs(accuracy_a - accuracy_c)
    return round(accuracy.item(),4), round(cali.item(),4)

In [100]:
#This function has a hyperparameter eta which is the size of the regulating term in the loss function.
#The fit method will give us the accuracy and caliberation of the fitted model
class PRLR():
  
    def __init__(self, eta=0.0, epochs=100, lr = 0.01):
        super(PRLR, self).__init__()
        self.eta = eta
        self.epochs = epochs
        self.lr = lr
        
    def fit(self,X_train_a,y_train_a,X_train_c,y_train_c,
            X_test_a, y_test_a, X_test_c, y_test_c):     
        model_a = LogisticRegression(X_train_a)
        model_c = LogisticRegression(X_train_c)
        criterion = nn.BCELoss(reduction='sum')
        PI = PRLoss(eta=self.eta)
        epochs = self.epochs
        #L2 regularization
        optimizer = t.optim.Adam(list(model_a.parameters())+ list(model_c.parameters()), self.lr, weight_decay=1e-5)
        
        for epoch in range(epochs):
            model_a.train()
            model_c.train()
            optimizer.zero_grad()
            output_a = model_a(X_train_a)
            output_c = model_c(X_train_c)
            logloss = criterion(output_a, y_train_a)+ criterion(output_c, y_train_c)
            PIloss = PI.forward(output_a,output_c)
            loss = PIloss +logloss
            loss.backward()
            optimizer.step()
            
        model_a.eval()
        model_c.eval()
        accuracy, calibration = metrics_cal(model_a,model_c,X_test_a, y_test_a, X_test_c, y_test_c)
        return accuracy, calibration

In [101]:
#Implement the function at eta=0.7
PR = PRLR(eta = 0.7, epochs = 100, lr = 1e-04)
PR.fit(X_train_afram,y_train_afram,X_train_cau,y_train_cau, X_test_afram, y_test_afram, X_test_cau, y_test_cau)

(0.5192, 0.2644)

In [103]:
#Tuning the hyperparameter to find the best model
eta=np.linspace(0, 100, num=1000)
hist_acc=np.zeros(1000)
hist_cal=np.zeros(1000)
for i in range(1000):
    PR = PRLR(eta = eta[i], epochs = 100, lr = 1e-04)
    hist_acc[i]=PR.fit(X_train_afram,y_train_afram,X_train_cau,y_train_cau, X_test_afram, y_test_afram, X_test_cau, y_test_cau)[0]
    hist_cal[i]=PR.fit(X_train_afram,y_train_afram,X_train_cau,y_train_cau, X_test_afram, y_test_afram, X_test_cau, y_test_cau)[1]

In [104]:
np.argmax(hist_acc)
best_eta=eta[np.argmax(hist_acc)]
#the best parameter for the model
best_eta

56.35635635635636

In [105]:
PR = PRLR(eta = best_eta, epochs = 100, lr = 1e-04)
PR.fit(X_train_afram,y_train_afram,X_train_cau,y_train_cau, X_test_afram, y_test_afram, X_test_cau, y_test_cau)

(0.5572, 0.1117)